In [4]:
import os
import sys
import torch
from torch.nn import functional as F
import numpy as np
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

def load_dataset(test_sen=None):

    """
    tokenizer : Breaks sentences into a list of words. If sequential=False, no tokenization is applied
    Field : A class that stores information about the way of preprocessing
    fix_length : An important property of TorchText is that we can let the input to be variable length, and TorchText will
                 dynamically pad each sequence to the longest sequence in that "batch". But here we are using fi_length which
                 will pad each sequence to have a fix length of 200.
                 
    build_vocab : It will first make a vocabulary or dictionary mapping all the unique words present in the train_data to an
                  idx and then after it will use GloVe word embedding to map the index to the corresponding word embedding.
                  
    vocab.vectors : This returns a torch tensor of shape (vocab_size x embedding_dim) containing the pre-trained word embeddings.
    BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
    
    """
    
    tokenize = lambda x: x.split()
    TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
    LABEL = data.LabelField()
    fields = [('boilerplate',TEXT),('label', LABEL)]


    #loading dataset
    training_data=data.TabularDataset(path = 'processed.csv',format = 'csv',fields = fields,skip_header = True)

    train_data, test_data = training_data.split(split_ratio=0.32)
    
    TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
    LABEL.build_vocab(train_data)

    word_embeddings = TEXT.vocab.vectors
    print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
    print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
    print ("Label Length: " + str(len(LABEL.vocab)))

    train_data, valid_data = train_data.split() # Further splitting of training_data to create new training_data & validation_data
    train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.boilerplate), repeat=False, shuffle=True)

    vocab_size = len(TEXT.vocab)

    return TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

class LSTMClassifier(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(LSTMClassifier, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		"""
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)# Initializing the look-up table.
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False) # Assigning the look-up table to the pre-trained GloVe word embedding.
		self.lstm = nn.LSTM(embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		
	def forward(self, input_sentence, batch_size=None):
	
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for positive & negative class which receives its input as the final_hidden_state of the LSTM
		final_output.shape = (batch_size, output_size)
		
		"""
		
		''' Here we will map all the indexes present in the input sequence to the corresponding word vector using our pre-trained word_embeddings.'''
		input = self.word_embeddings(input_sentence) # embedded input of shape = (batch_size, num_sequences,  embedding_length)
		input = input.permute(1, 0, 2) # input.size() = (num_sequences, batch_size, embedding_length)
		if batch_size is None:
			h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) # Initial hidden state of the LSTM
			c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) # Initial cell state of the LSTM
		else:
			h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
		final_output = self.label(final_hidden_state[-1]) # final_hidden_state.size() = (1, batch_size, hidden_size) & final_output.size() = (batch_size, output_size)
		
		return final_output

In [8]:
import os
import time
import torch
# import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
# import load_data
# from models.LSTM import LSTMClassifier

TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_dataset()

def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        boilerplate = batch.boilerplate[0]
        target = batch.label
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            boilerplate = boilerplate.cuda()
            target = target.cuda()
        if (boilerplate.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(boilerplate)
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            boilerplate = batch.boilerplate[0]
            if (boilerplate.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                boilerplate = boilerplate.cuda()
                target = target.cuda()
            prediction = model(boilerplate)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)
	

# learning_rate = 2e-5
learning_rate = .01
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300

model = LSTMClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
loss_fn = F.cross_entropy
# loss_fn = nn.CrossEntropyLoss()


for epoch in range(30):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')



Length of Text Vocabulary: 29846
Vector size of Text Vocabulary:  torch.Size([29846, 300])
Label Length: 2
Epoch: 01, Train Loss: 0.603, Train Acc: 63.16%, Val. Loss: 0.564506, Val. Acc: 72.17%
Epoch: 02, Train Loss: 0.507, Train Acc: 76.20%, Val. Loss: 0.534926, Val. Acc: 72.17%
Epoch: 03, Train Loss: 0.509, Train Acc: 74.87%, Val. Loss: 0.514947, Val. Acc: 72.02%
Epoch: 04, Train Loss: 0.492, Train Acc: 75.53%, Val. Loss: 0.524570, Val. Acc: 72.32%
Epoch: 05, Train Loss: 0.457, Train Acc: 78.39%, Val. Loss: 0.511250, Val. Acc: 72.47%
Epoch: 06, Train Loss: 0.424, Train Acc: 80.25%, Val. Loss: 0.529020, Val. Acc: 72.47%
Epoch: 07, Train Loss: 0.380, Train Acc: 82.31%, Val. Loss: 0.549017, Val. Acc: 72.02%
Epoch: 08, Train Loss: 0.350, Train Acc: 83.31%, Val. Loss: 0.592984, Val. Acc: 70.09%
Epoch: 09, Train Loss: 0.360, Train Acc: 82.78%, Val. Loss: 0.633140, Val. Acc: 65.62%
Epoch: 10, Train Loss: 0.316, Train Acc: 84.71%, Val. Loss: 0.602505, Val. Acc: 73.21%
Epoch: 11, Train Loss: 

In [39]:
prediction = model(boilerplate)

NameError: ignored

In [31]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [38]:
import os
import time
import torch
# import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
# import load_data
# from models.LSTM import LSTMClassifier

TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_dataset()

def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        boilerplate = batch.boilerplate[0]
        target = batch.label
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            boilerplate = boilerplate.cuda()
            target = target.cuda()
        if (boilerplate.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(boilerplate)
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()

    
    y_pred = []
    y_true = []
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            boilerplate = batch.boilerplate[0]
            if (boilerplate.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                boilerplate = boilerplate.cuda()
                target = target.cuda()
            
            prediction = model(boilerplate)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

            y_true.extend(target.tolist())
            a=torch.max(prediction, 1)[1].view(target.size()).data
            y_pred.extend(a.tolist())

    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=[0,1], digits=4))
   
    print(len(y_pred))
    print(len(y_true))
    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)
	

# learning_rate = 2e-5
learning_rate = .01
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300

model = LSTMClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)

class F1_Loss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, 2).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)
        
        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        return 1 - f1.mean()


loss_fn = F1_Loss().cuda()

for epoch in range(25):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')



Length of Text Vocabulary: 29846
Vector size of Text Vocabulary:  torch.Size([29846, 300])
Label Length: 2
Classification Report:
              precision    recall  f1-score   support

           0     0.8312    0.7399    0.7829       346
           1     0.7289    0.8231    0.7732       294

    accuracy                         0.7781       640
   macro avg     0.7800    0.7815    0.7780       640
weighted avg     0.7842    0.7781    0.7784       640

640
640
Epoch: 01, Train Loss: 0.332, Train Acc: 67.55%, Val. Loss: 0.230925, Val. Acc: 74.11%
Classification Report:
              precision    recall  f1-score   support

           0     0.7865    0.7775    0.7820       346
           1     0.7416    0.7517    0.7466       294

    accuracy                         0.7656       640
   macro avg     0.7641    0.7646    0.7643       640
weighted avg     0.7659    0.7656    0.7657       640

640
640
Epoch: 02, Train Loss: 0.201, Train Acc: 78.39%, Val. Loss: 0.240988, Val. Acc: 72.92%
Cla

In [ ]:
import pandas as pd
test=pd.read_csv("processed_test.csv")
test.head()

In [55]:
final=[]
for i in range(10):
  # test_sen1 = "This is one of the best creation of Nolan. I can say, it's his magnum opus. Loved the soundtrack and especially those creative dialogues."
  # test_sen2 = "Ohh, such a ridiculous movie. Not gonna recommend it to anyone. Complete waste of time and money."
  test_sen1=test["boilerplate"][i]
  
  test_sen1 = TEXT.preprocess(test_sen1)
  test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]

  # test_sen2 = TEXT.preprocess(test_sen2)
  # test_sen2 = [[TEXT.vocab.stoi[x] for x in test_sen2]]

  test_sen = np.asarray(test_sen1)
  test_sen = torch.LongTensor(test_sen)
  test_tensor = Variable(test_sen, volatile=True)
  test_tensor = test_tensor.cuda()
  model.eval()
  output = model(test_tensor, 1)
  out = F.softmax(output, 1)

  final.append(torch.argmax(out[0]).tolist())
  # if (torch.argmax(out[0]) == 1):
  #     print ("Sentiment: Positive")
  # else:
  #     print ("Sentiment: Negative")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app


In [58]:
print(final)

[0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
